In [30]:
import requests
import json

HOST = "http://10.35.0.112:8000/"
API_CANCLED = "canclled"
API_PNR_RANKING = "pnr_ranking?flight_id="
API_ALT_FLIGHT="alt_flight?flight_id="

def get_cancled():
    response = requests.request("GET", HOST + API_CANCLED, headers={}, data={})
    return json.loads(response.text)

def get_pnrs(flight_id):
    response = requests.request("GET", HOST + API_PNR_RANKING + str(flight_id), headers={}, data={})
    return json.loads(response.text)

def get_alt_flights(flight_id):
    response = requests.request("GET", HOST + API_ALT_FLIGHT + str(flight_id), headers={}, data={})
    return json.loads(response.text)

In [31]:
cancel=get_cancled()

In [32]:
cancel['data'][0]
map_cabin={ 'F':'F','P':'F','C':'B','J':'B','Z':'B','Q':'P','R':'P','S':'P','T':'P','H':'P','M':'P',
          'Y':'E', 'A':'E', 'B':'E', 'D':'E', 'E':'E', 'G':'E', 'I':'E', 'K':'E', 'L':'E', 'N':'E', 'O':'E',
           'U':'E', 'V':'E','W':'E','X':'E'}

In [86]:
get_alt_flights('c53865e9')

ConnectionError: HTTPConnectionPool(host='10.35.0.112', port=8000): Max retries exceeded with url: /alt_flight?flight_id=c53865e9 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000251D2C4EEF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [53]:
pnr={}
alt_flight={}
for f in cancel['data']:
    pnr[f['flight_id']]=get_pnrs(f['flight_id'])['data']
    alt=get_alt_flights(f['flight_id'])['data']
    f2=[]
    for flight in alt:
        f3={}
        f3['flight_id']=flight['flight_id']
        f3['F']=0
        f3['B']=0
        f3['P']=0
        f3['E']=0
#         print()
        for key in map_cabin:
            if key in flight['avilable_seats']:
                f3[map_cabin[key]]+=flight['avilable_seats'][key]
        f2.append(f3)
    alt_flight[f['flight_id']]=f2

In [213]:
for i in alt_flight:
    print(alt_flight[i])
    break
    

[{'flight_id': '6246013f', 'F': 9, 'B': 12, 'P': 8, 'E': 23}, {'flight_id': 'a3338dd3', 'F': 4, 'B': 6, 'P': 7, 'E': 33}, {'flight_id': '79791ce3', 'F': 3, 'B': 6, 'P': 15, 'E': 22}, {'flight_id': '487a3129', 'F': 0, 'B': 13, 'P': 10, 'E': 31}, {'flight_id': '82ab029c', 'F': 6, 'B': 3, 'P': 7, 'E': 18}, {'flight_id': '93d3ede0', 'F': 3, 'B': 13, 'P': 5, 'E': 27}, {'flight_id': '6e7e9e93', 'F': 5, 'B': 10, 'P': 10, 'E': 22}, {'flight_id': '383a9958', 'F': 7, 'B': 6, 'P': 7, 'E': 33}]


In [273]:
used_seat={}

for i in alt_flight:
    for j in alt_flight[i]:
        used_seat[j['flight_id']]={'F':0,'B':0,'P':0,'E':0}
#             print(j)
        
used_seat

{'6246013f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'a3338dd3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '79791ce3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '487a3129': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '82ab029c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '93d3ede0': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '6e7e9e93': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '383a9958': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '3edfe26b': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'e37c018c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'ed6fcfe5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'ec0634ec': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9e5265d4': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd760ce08': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd6d4c356': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '6e205e49': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd9707350': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '98b536f8': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd7e33413': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '1682c749': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'c11ccb3e': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '658c1364': 

In [274]:
pnr

{'c53865e9': [{'pnr': 'e36140',
   'score': 2700,
   'class': 'F',
   'pax': 4,
   'flight_id': 'c53865e9'},
  {'pnr': '9c112d',
   'score': 2550,
   'class': 'P',
   'pax': 5,
   'flight_id': 'c53865e9'},
  {'pnr': 'a53d26',
   'score': 2350,
   'class': 'P',
   'pax': 1,
   'flight_id': 'c53865e9'},
  {'pnr': 'ddb1f4',
   'score': 2300,
   'class': 'P',
   'pax': 4,
   'flight_id': 'c53865e9'},
  {'pnr': '7d7296',
   'score': 2250,
   'class': 'F',
   'pax': 1,
   'flight_id': 'c53865e9'},
  {'pnr': 'd7eff6',
   'score': 2250,
   'class': 'P',
   'pax': 1,
   'flight_id': 'c53865e9'},
  {'pnr': 'c5fa12',
   'score': 2250,
   'class': 'E',
   'pax': 1,
   'flight_id': 'c53865e9'},
  {'pnr': '99919b',
   'score': 2200,
   'class': 'F',
   'pax': 4,
   'flight_id': 'c53865e9'},
  {'pnr': '19d7bd',
   'score': 2200,
   'class': 'F',
   'pax': 4,
   'flight_id': 'c53865e9'},
  {'pnr': '2f2f86',
   'score': 2150,
   'class': 'P',
   'pax': 3,
   'flight_id': 'c53865e9'},
  {'pnr': '80104a'

In [275]:
needed_seat=[]
for i in pnr:
    alloc={}
    for j in pnr[i]:
        alloc=j;
        alloc['flight_id']=i
        needed_seat.append(alloc)

In [276]:
sorted_pnr = sorted(needed_seat, key=lambda x: x['score'],reverse=True)

In [277]:
allocated={}
for i in sorted_pnr:
    allocated[i['pnr']]='NULL'

In [281]:
index_pnr=0
flight_index={i : {'F':0,'B':0,'P':0,'E':0} for i in pnr}
# flight_index
used_seat

{'6246013f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'a3338dd3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '79791ce3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '487a3129': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '82ab029c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '93d3ede0': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '6e7e9e93': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '383a9958': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '3edfe26b': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'e37c018c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'ed6fcfe5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'ec0634ec': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9e5265d4': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd760ce08': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd6d4c356': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '6e205e49': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd9707350': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '98b536f8': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'd7e33413': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '1682c749': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'c11ccb3e': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '658c1364': 

In [282]:
# ik=0
for i in sorted_pnr:
#     ik+=1
#     if(ik>5):
#         break
#     print(i)
    cabin=map_cabin[i['class']]
    ind = 0
#     print(used_seat)
    while(ind<len(alt_flight[i['flight_id']])):
        
        max_seat=alt_flight[i['flight_id']][ind][cabin]
        alt = alt_flight[i['flight_id']][ind]['flight_id']
        print(max_seat)
        print(ind)
        print(alt)
#         print(used_seat)
        
        if(used_seat[alt][cabin]+i['pax']<=max_seat):
            used_seat[alt][cabin]+=i['pax'];
            allocated[i['pnr']]=alt
            print("allovc")
            break            
        
        ind+=1
        

2
0
634e5d5c
4
1
ee6ac073
allovc
10
0
ad0d072f
allovc
2
0
71b1c03b
2
1
942faf88
0
2
581862fc
2
3
af5c5b8e
0
4
105b8907
0
5
a475fd41
12
6
17974838
allovc
0
0
2c4d7de2
0
1
e2cb187f
0
2
2b7346c8
0
3
819c02be
0
4
68a65b1c
0
5
12fe2023
0
6
415cf1f5
5
0
2c4d7de2
allovc
3
0
71b1c03b
3
1
942faf88
2
2
581862fc
3
3
af5c5b8e
5
4
105b8907
allovc
39
0
bcff4322
allovc
10
0
d4ce8a8e
allovc
2
0
2c4d7de2
4
1
e2cb187f
3
2
2b7346c8
5
3
819c02be
allovc
9
0
6246013f
allovc
4
0
e887392d
allovc
0
0
11ebf189
0
1
c02f44f6
0
2
e8b2438b
0
3
a3f11787
0
4
b295fd26
7
5
62169d28
allovc
5
0
21db4339
allovc
4
0
e887392d
8
1
5c6e92d2
allovc
10
0
755a659b
allovc
10
0
b3a5d715
allovc
10
0
4aa6cb43
allovc
0
0
11ebf189
0
1
c02f44f6
0
2
e8b2438b
0
3
a3f11787
0
4
b295fd26
7
5
62169d28
2
6
51a71fb1
13
7
95e5c86a
allovc
0
0
11ebf189
0
1
c02f44f6
0
2
e8b2438b
0
3
a3f11787
0
4
b295fd26
7
5
62169d28
allovc
6
0
9d08aad9
allovc
3
0
a3658f53
allovc
10
0
755a659b
allovc
9
0
6246013f
allovc
5
0
21db4339
2
1
81f181b5
allovc
18
0
21db43

In [283]:
used_seat[alt]

{'F': 4, 'B': 0, 'P': 9, 'E': 22}

In [284]:
allocated

{'6be5cc': 'ee6ac073',
 'cde5e8': 'ad0d072f',
 '26c155': '17974838',
 'dd676b': 'NULL',
 '8fcd1e': '2c4d7de2',
 '17765d': '105b8907',
 'd8aab8': 'bcff4322',
 '72b3b1': 'd4ce8a8e',
 'b07182': '819c02be',
 'e36140': '6246013f',
 '33a7db': 'e887392d',
 '296ae4': '62169d28',
 '4b3deb': '21db4339',
 'f2087c': '5c6e92d2',
 '272f22': '755a659b',
 '4fb227': 'b3a5d715',
 '71ca17': '4aa6cb43',
 '6d0ec5': '95e5c86a',
 '71038a': '62169d28',
 'a1b102': '9d08aad9',
 '90b52c': 'a3658f53',
 '40e949': '755a659b',
 '9c112d': '6246013f',
 '59d8d2': '81f181b5',
 'c4bffc': '21db4339',
 '9b95c5': '755a659b',
 '4b0c46': 'd4ce8a8e',
 '9cc845': '17974838',
 '76aa4a': '9d08aad9',
 'eb0db0': '97ba4e4f',
 '79cd41': 'bcff4322',
 '4a73e9': 'f8c12a49',
 '84ed58': 'b3a5d715',
 '9152a5': '95e5c86a',
 'eb3706': '7e0b05a7',
 'bdf4bd': '3edfe26b',
 '5dcb8b': '81f181b5',
 '9bd290': 'a536214b',
 '5393d4': 'b3a5d715',
 '927ac6': '51a71fb1',
 'e0418d': 'd4ce8a8e',
 'beef87': '7e0b05a7',
 '6db321': 'e2cb187f',
 'f510d5': 'a9d

In [285]:
sum=0
for i in allocated: 
    if allocated[i]=='NULL':
        sum+=1

In [286]:
sum


228

In [288]:
len(allocated)

2545

In [ ]:
sorted